In [3]:
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import numpy as np
import facenet_model as fnm

In [4]:
target_shape = (112, 112)

In [5]:
facenet_model = fnm.get_facenet_model()

In [6]:
anchor_input = keras.layers.Input(name='anchor', shape=target_shape+(3,))
positive_input = keras.layers.Input(name='positive', shape=target_shape+(3,))
negative_input = keras.layers.Input(name='negative', shape=target_shape+(3,))

In [7]:
distances = fnm.TripletDistanceLayer()(
    facenet_model(anchor_input),
    facenet_model(positive_input),
    facenet_model(negative_input)
)

In [8]:
siamese_network = keras.models.Model(
    inputs=[anchor_input, positive_input, negative_input], outputs=distances
)

In [27]:
import os
import matplotlib.pyplot as plt
import cv2 as cv
import math

In [28]:
# X = utils.loadimgs("/home/nimahsn/Downloads/Compressed/CASIA-cropped")

In [31]:
n_epochs = 500
# batch_size = 32
base_path = '/home/nimahsn/Downloads/Compressed/CASIA-cropped/'
# images
dirs = os.listdir(base_path)
batch_size = 32
n_steps = math.ceil(len(dirs) / 32)
# n_steps = len(dirs)

In [39]:
def generate_batch():
    np.random.shuffle(dirs)
    batch = []
    i = 0
    for anchor_path in dirs:
        positives = os.listdir(base_path + anchor_path)
        p = np.random.choice(positives, size=2, replace=False)
        # print(base_path+anchor_path+p[0], flush=True)
        anchor = cv.resize(cv.imread(base_path+anchor_path+'/'+p[0]), dsize=[112,112])
        positive = cv.resize(cv.imread(base_path+anchor_path+'/'+p[1]), dsize=[112,112])
        n_path = anchor_path
        while n_path == anchor_path:
            n_path = np.random.choice(dirs, size=1)[0]
        # print(np.random.choice(os.listdir(base_path+n_path), size=1))
        n_path = base_path + n_path + '/'+np.random.choice(os.listdir(base_path+n_path), size=1)[0]
        negative = cv.resize(cv.imread(n_path), dsize=[112, 112])
        batch.append((anchor, positive, negative))
        i+=1
        if i == batch_size:
            yield np.array(batch)
            batch = []
            i = 0
    yield np.array(batch)



In [39]:
for epoch in range(n_epochs):
    print('epoch {} / {}'.format(epoch, n_epochs))
    step = 0
    for X_batch in generate_batch():
        step += 1
        with tf.GradientTape() as tape:
            y_pred = siamese_network